# Radial Basis Function

Σκοπός της παρούσας άσκησης είναι η πρόβλεψη ως προς το αν ένας/μία πελάτης/ισσα μίας τράπεζας θα κάνει τη χρήση της προθεσμιακής κατάθεσης ή όχι. Αυτή η πρόβλεψη θα εξαχθεί με βάση συγκεκριμένα δημογραφικά χαρακτηριστικά του/της
όπως η ηλικία, η εργασία, η εκπαίδευση, η οικογενειακή κατάσταση κ.λπ.
Για να το επιτύχουμε αυτό θα κάνουμε χρήση μίας εφαρμογής βασισμένη σε RBF. Στο κρυφό επίπεδο των νευρώνων υπολογίζονται τα κέντρα των clusters με τη χρήση του αλγορίθμου k-means.




**Βήμα 1ο**: Αρχικά φορτώνουμε τις απαράιτητες βιβλιοθήκες

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import math
from sklearn.metrics import accuracy_score

**Βήμα 2ο**: διάβαζουμε το αρχείο bank-full.csv και απομονώνουμε τις στήλες με τα χαρακτηριστικά που θα χρησιμοποιήσουμε (εργασία, αν έχει δανειακές υποχρεώσεις κλπ). Το αρχείο είναι διαθέσιμο [εδώ](https://raw.githubusercontent.com/nkostopoulos/StochasticsLabPublic/master/lab8/bank-full.csv) και θα πρέπει να το φορτώσετε στο colab.

In [2]:
dataset = pd.read_csv('bank-full.csv',sep=None)

<ipython-input-2-f217a134c1fa>:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support sep=None with delim_whitespace=False; you can avoid this warning by specifying engine='python'.
  dataset = pd.read_csv('bank-full.csv',sep=None)


In [3]:
dataset.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


In [4]:
col_to_use = ['age','balance','day','duration','campaign','pdays','previous']
data = dataset.drop(col_to_use,axis=1)

In [5]:
data.head()

,job,marital,education,default,housing,loan,contact,month,poutcome,y
0,management,married,tertiary,no,yes,no,unknown,may,unknown,no
1,technician,single,secondary,no,yes,no,unknown,may,unknown,no
2,entrepreneur,married,secondary,no,yes,yes,unknown,may,unknown,no
3,blue-collar,married,unknown,no,yes,no,unknown,may,unknown,no
4,unknown,single,unknown,no,no,no,unknown,may,unknown,no


In [6]:
data = data.apply(LabelEncoder().fit_transform)

In [7]:
data.head()

,job,marital,education,default,housing,loan,contact,month,poutcome,y
0,4,1,2,0,1,0,2,8,3,0
1,9,2,1,0,1,0,2,8,3,0
2,2,1,1,0,1,1,2,8,3,0
3,1,1,3,0,1,0,2,8,3,0
4,11,2,3,0,0,0,2,8,3,0


In [8]:
data['job'].unique()

array([ 4,  9,  2,  1, 11,  5,  0,  7,  6, 10,  3,  8])

In [9]:
dataset['job'].unique()

array(['management', 'technician', 'entrepreneur', 'blue-collar',
       'unknown', 'retired', 'admin.', 'services', 'self-employed',
       'unemployed', 'housemaid', 'student'], dtype=object)

In [10]:
data_rest = dataset[col_to_use]
data_rest.head()

,age,balance,day,duration,campaign,pdays,previous
0,58,2143,5,261,1,-1,0
1,44,29,5,151,1,-1,0
2,33,2,5,76,1,-1,0
3,47,1506,5,92,1,-1,0
4,33,1,5,198,1,-1,0


In [11]:
dataset2 = pd.concat([data_rest,data],axis=1)

In [12]:
dataset2.head()

,age,balance,day,duration,campaign,pdays,previous,job,marital,education,default,housing,loan,contact,month,poutcome,y
0,58,2143,5,261,1,-1,0,4,1,2,0,1,0,2,8,3,0
1,44,29,5,151,1,-1,0,9,2,1,0,1,0,2,8,3,0
2,33,2,5,76,1,-1,0,2,1,1,0,1,1,2,8,3,0
3,47,1506,5,92,1,-1,0,1,1,3,0,1,0,2,8,3,0
4,33,1,5,198,1,-1,0,11,2,3,0,0,0,2,8,3,0


**Βήμα 3ο**: Σπάμε στο dataset σε train και test dataset και κάνουμε scale με τη χρήση της συνάρτησης StandardScaler()

In [13]:
X= dataset2.drop('y',axis=1)
y= dataset2['y']

X_train, X_test, y_train, y_test= train_test_split(X,y, test_size= 0.33, random_state= 4)

In [14]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [15]:
X_train

array([[ 3.39736083, -0.45852779,  0.14087303, ..., -0.71332324,
        -1.50513854,  0.44672656],
       [-0.27605947,  2.43941798,  0.50067953, ...,  0.40010942,
        -1.83733874, -2.58685457],
       [ 0.66584317, -0.13326522, -0.93854645, ..., -0.71332324,
         1.48466323, -0.56446715],
       ...,
       [-0.74701079, -0.22619739,  0.62061503, ..., -0.71332324,
         1.15246303,  0.44672656],
       [ 1.60774581, -0.42275894, -1.53822394, ...,  1.51354209,
         0.15586244,  0.44672656],
       [ 0.28908211, -0.13627382,  1.58009901, ..., -0.71332324,
        -0.17633776,  0.44672656]])

**Βήμα 4ο**: Ορίζουμε τα κέντρα των νευρώνων με τη χρήση του k-means. Αρχικά τρέχει ο αλγόριθμος k-means για να βρούμε τα κέντρα και την τυπική απόκλιση των clusters.

In [16]:
K_cent= 8
km= KMeans(n_clusters= K_cent, max_iter= 100)
km.fit(X_train)
cent= km.cluster_centers_

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [17]:
max=0
for i in range(K_cent):
	for j in range(K_cent):
		d= np.linalg.norm(cent[i]-cent[j])
		if(d> max):
			max= d
d= max

sigma= d/math.sqrt(2*K_cent)

Βήμα 5ο: Δημιουργούμε τον πίνακα σχέσεων F (γραμμικός συνδυασμός των Κ συναρτήσεων βάσης φj(x).

In [18]:
shape= X_train.shape
row= shape[0]
column= K_cent
F= np.empty((row,column), dtype= float)

In [19]:
for i in range(row):
  for j in range(column):
    dist= np.linalg.norm(X_train[i]-cent[j])
    F[i][j]= math.exp(-math.pow(dist,2)/math.pow(2*sigma,2))

**Βήμα 6ο**: Βρίσκουμε τον πίνακα βάρους W για την εκπαίδευση του δικτύου

In [20]:
FTG= np.dot(F.T,F)
FTG_inv= np.linalg.inv(FTG)
fac= np.dot(FTG_inv,F.T)
W= np.dot(fac,y_train)

**Βήμα 7ο**: Ρύθμιση του πίνακα F για το test dataset

In [21]:
row= X_test.shape[0]
column= K_cent
F_test= np.empty((row,column), dtype= float)
for i in range(row):
	for j in range(column):
		dist= np.linalg.norm(X_test[i]-cent[j])
		F_test[i][j]= math.exp(-math.pow(dist,2)/math.pow(2*sigma,2))

**Βήμα 8ο**: Ακρίβεια πρόβλεψης στο test dataset

In [22]:
prediction= np.dot(F_test,W)
prediction= 0.5*(np.sign(prediction-0.5)+1)

score= accuracy_score(prediction,y_test)
print(score.mean())

0.8876675603217158


## Ερωτήσεις

**Ερώτηση 1**: Να αναφέρετε επιγραμματικά με ποιον τρόπο λειτουργεί ένα νευρωνικό δίκτυο RBF

**Ερώτηση 2**:Με ποιούς τρόπους μπορούμε να υπολογίσουμε τα βάρη στα hidden layers; Ποιες διαφορές υπάρχουν στους τρόπους αυτούς; Ποιος παρέχει τη καλύτερη δυνατή λύση;

**Ερώτηση 3**: Επιχειρήστε να αλλάξετε την τιμή του k σε 10 και στη συνέχεια σε 50. Τι παρατηρείτε ως προς την ακρίβεια των αποτελεσμάτων;

ΑΠΑΝΤΗΣΕΙΣ:


ΕΡΩΤΗΣΗ 1:

Τα νευρωνικά δίκτυα RBF είναι εννοιολογικά παρόμοια με τα μοντέλα K-Nearest Neighbor (k-NN), αν και η υλοποίηση και των δύο μοντέλων είναι εντελώς διαφορετική. Η θεμελιώδης ιδέα των συναρτήσεων ακτινικής βάσης είναι ότι η προβλεπόμενη τιμή στόχου ενός στοιχείου είναι πιθανό να είναι ίδια με άλλα στοιχεία με κοντινές τιμές μεταβλητών πρόβλεψης. Ένα δίκτυο RBF τοποθετεί έναν ή πολλούς νευρώνες RBF στο χώρο που περιγράφεται από τις μεταβλητές πρόβλεψης. Ο χώρος έχει πολλαπλές διαστάσεις που αντιστοιχούν στον αριθμό των προγνωστικών μεταβλητών που υπάρχουν (target-y). Υπολογίζουμε την Ευκλείδεια απόσταση από το αξιολογούμενο σημείο έως το κέντρο κάθε νευρώνα. Μια ακτινική συνάρτηση βάσης (Radial Basisi Function-RBF), επίσης γνωστή ως συνάρτηση πυρήνα, εφαρμόζεται στην απόσταση για τον υπολογισμό του βάρους(επιρροή) κάθε νευρώνα. Το όνομα της συνάρτησης ακτινικής βάσης προέρχεται από την απόσταση ακτίνας, η οποία είναι το όρισμα της συνάρτησης. Βάρος = RBF[απόσταση) Όσο μεγαλύτερη είναι η απόσταση ενός νευρώνα από το σημείο που αξιολογείται, τόσο λιγότερη επιρροή (βάρος) έχει.

Δηλαδή, η RBF έχει  τόσους νευρώνες όσο και το πλήθος των μεταβλητών πρόβλεψης(y). Αν ωστόσο συμβεί, να βρίσκονται 3 νευρώνες σε έναν χώρο με 2 μεταβλητές πρόβλεψης, μπορούμε να προβλέψουμε την τιμή από τις συναρτήσεις RBF. Μπορούμε να υπολογίσουμε την καλύτερη προβλεπόμενη τιμή για το νέο σημείο προσθέτοντας τις τιμές εξόδου των συναρτήσεων RBF πολλαπλασιαζόμενες με τα βάρη που υποβάλλονται σε επεξεργασία για κάθε νευρώνα.

Είσοδος συνάρτησης: Είναι ένα  διάνυσμα n-διαστάσεων που προσπαθούμε να ταξινομήσουμε. Ολόκληρο το διάνυσμα εισόδου παρουσιάζεται σε κάθε έναν από τους νευρώνες RBF.

Κάθε νευρώνας RBF αποθηκεύει ένα πρωτότυπο διάνυσμα (επίσης γνωστό ως το κέντρο του νευρώνα) από τα διανύσματα του συνόλου εκπαίδευσης. Ένας νευρώνας RBF συγκρίνει το διάνυσμα εισόδου με το πρωτότυπό του και εξάγει μια τιμή μεταξύ 0 και 1 ως μέτρο ομοιότητας. Εάν μια είσοδος είναι ίδια με το πρωτότυπο, η έξοδος του νευρώνα θα είναι 1. Καθώς η διαφορά εισόδου και πρωτοτύπου μεγαλώνει, η έξοδος πέφτει εκθετικά προς το 0. Η τιμή απόκρισης ονομάζεται επίσης τιμή ενεργοποίησης.

Έξοδος: Η έξοδος του δικτύου περιλαμβάνει ένα σύνολο κόμβων για κάθε κατηγορία που προσπαθούμε να ταξινομήσουμε. Κάθε κόμβος εξόδου υπολογίζει μια βαθμολογία για τη σχετική κατηγορία. Γενικά, λαμβάνουμε μια απόφαση ταξινόμησης εκχωρώντας τα δεδομένα στην κατηγορία με την υψηλότερη βαθμολογία.

Η βαθμολογία υπολογίζεται με βάση ένα σταθμισμένο άθροισμα των τιμών ενεργοποίησης από όλους τους νευρώνες RBF. Συνήθως δίνει θετικό βάρος στον νευρώνα RBF που ανήκει στην κατηγορία του και αρνητικό βάρος σε άλλους. Κάθε κόμβος εξόδου έχει το δικό του σύνολο βαρών.  

ΕΡΩΤΗΣΗ 2:

Το κρυφό στρώμα περιέχει έναν μεταβλητό αριθμό νευρώνων (ο ιδανικός αριθμός που καθορίζεται από τη διαδικασία εκπαίδευσης). Κάθε νευρώνας περιλαμβάνει μια συνάρτηση ακτινικής βάσης που επικεντρώνεται σε ένα σημείο. Ο αριθμός των διαστάσεων συμπίπτει με τον αριθμό των μεταβλητών πρόβλεψης. Η ακτίνα ή η εξάπλωση της συνάρτησης RBF μπορεί να διαφέρει για κάθε διάσταση.

Όταν ένα x διάνυσμα τιμών εισόδου τροφοδοτείται από το επίπεδο εισόδου, ένας κρυφός νευρώνας υπολογίζει την Ευκλείδεια απόσταση μεταξύ της δοκιμαστικής περίπτωσης και του κεντρικού σημείου του νευρώνα. Στη συνέχεια, εφαρμόζει τη συνάρτηση πυρήνα χρησιμοποιώντας τις τιμές spread. Η προκύπτουσα τιμή τροφοδοτείται στο επίπεδο άθροισης.

Η λειτουργεία που επιτελεί το επίπεδο άθροισης είναι να λαμβάνει την τιμή από το κρυφό στρώμα και η τιμή αυτή  πολλαπλασιάζεται με ένα βάρος που σχετίζεται με τον νευρώνα και περνά στο άθροισμα. Εδώ προστίθενται οι σταθμισμένες τιμές και το άθροισμα παρουσιάζεται ως έξοδος του δικτύου. Τα προβλήματα ταξινόμησης έχουν μία έξοδο ανά κατηγορία στόχο, η τιμή είναι η πιθανότητα ότι η περίπτωση που αξιολογήθηκε έχει αυτήν την κατηγορία.

Σε ό,τι έχει να κάνει με τον υπολογισμό των βαρών του κρυφού επιπέδου, δηλαδή την ίδια την εκπαίδευση του δικτύου, υπάρχουν διάφορες τεχνικές.
Συγκεκριμένα, μπορούμε να χρησιμοποιήσουμε τις εξής μαθόδους:

Ευκλείδια απόσταση: Περιγράφτηκε παραπάνω

Τυχαία Αρχικοποίηση (Random Initialization): Σε αυτήν τη μέθοδο, τα βάρη αρχικοποιούνται τυχαία από μια κατανομή, όπως η κανονική κατανομή ή η ομοιόμορφη κατανομή. Αυτή η μέθοδος είναι απλή και γρήγορη, αλλά δεν εγγυάται τη σύγκλιση σε ικανοποιητικές λύσεις.


Xαριστική Αρχικοποίηση (He Initialization): Αυτή η μέθοδος αρχικοποιεί τα βάρη με τυχαίους αριθμούς που είναι κατανεμημένοι με βάση τη χαριστική των εισόδων του επιπέδου. Αυτό βοηθά στην αποφυγή του προβλήματος της ασταθής έκρηξης ή εξαφάνισης των κλίμακων κατά την εκπαίδευση του μοντέλου.

Μεταφορά Μάθησης (Transfer Learning): Μπορούμε να χρησιμοποιήσουμε βάρη που έχουν εκπαιδευτεί σε ένα άλλο μοντέλο που έχει εκπαιδευτεί σε παρόμοια δεδομένα. Αυτή η μέθοδος είναι χρήσιμη όταν έχουμε περιορισμένο όγκο δεδομένων εκπαίδευσης.

Εκπαίδευση από το Μηδέν (Training from Scratch): Σε αυτήν την προσέγγιση, τα βάρη αρχικοποιούνται τυχαία και το μοντέλο εκπαιδεύεται από την αρχή χωρίς προεκπαιδευμένα βάρη.

Όπως γίνεται αντιληπτό, η βασική διαφορά ανάμεσα στις παραπάνω μεθόδους έγκειται στην αρχικοποίηση των ακτινικών συναρτήσεων βάσης, η οποία γίνεται με τυχαίο τρόπο στις περισσότερες από τις παραπάνω περιπτώσεις. Προφανώς, ο υπολογισμός της ευκλείδιας απόστασης, που είναι και η μέθοδος που παρουσιάζει το παρόν notebook, είναι η ιδανικότερη, αφού μια κακή αρχικοποίηση των παραμέτρων των ακτινικών συναρτήσεων βάσης μπορεί να επηρεάσει αρνητικά την εκπαίδευση του δικτύου

ΕΡΩΤΗΣΗ 3:

In [26]:
K_cent= 10
km= KMeans(n_clusters= K_cent, max_iter= 100)
km.fit(X_train)
cent= km.cluster_centers_
max=0
for i in range(K_cent):
	for j in range(K_cent):
		d= np.linalg.norm(cent[i]-cent[j])
		if(d> max):
			max= d
d= max

sigma= d/math.sqrt(2*K_cent)


shape= X_train.shape
row= shape[0]
column= K_cent
F= np.empty((row,column), dtype= float)

for i in range(row):
  for j in range(column):
    dist= np.linalg.norm(X_train[i]-cent[j])
    F[i][j]= math.exp(-math.pow(dist,2)/math.pow(2*sigma,2))

FTG= np.dot(F.T,F)
FTG_inv= np.linalg.inv(FTG)
fac= np.dot(FTG_inv,F.T)
W= np.dot(fac,y_train)

row= X_test.shape[0]
column= K_cent
F_test= np.empty((row,column), dtype= float)
for i in range(row):
	for j in range(column):
		dist= np.linalg.norm(X_test[i]-cent[j])
		F_test[i][j]= math.exp(-math.pow(dist,2)/math.pow(2*sigma,2))


prediction= np.dot(F_test,W)
prediction= 0.5*(np.sign(prediction-0.5)+1)

score= accuracy_score(prediction,y_test)
print(score.mean())

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


0.8912198391420911


In [24]:
K_cent= 50
km= KMeans(n_clusters= K_cent, max_iter= 100)
km.fit(X_train)
cent= km.cluster_centers_
max=0
for i in range(K_cent):
	for j in range(K_cent):
		d= np.linalg.norm(cent[i]-cent[j])
		if(d> max):
			max= d
d= max

sigma= d/math.sqrt(2*K_cent)


shape= X_train.shape
row= shape[0]
column= K_cent
F= np.empty((row,column), dtype= float)

for i in range(row):
  for j in range(column):
    dist= np.linalg.norm(X_train[i]-cent[j])
    F[i][j]= math.exp(-math.pow(dist,2)/math.pow(2*sigma,2))

FTG= np.dot(F.T,F)
FTG_inv= np.linalg.inv(FTG)
fac= np.dot(FTG_inv,F.T)
W= np.dot(fac,y_train)

row= X_test.shape[0]
column= K_cent
F_test= np.empty((row,column), dtype= float)
for i in range(row):
	for j in range(column):
		dist= np.linalg.norm(X_test[i]-cent[j])
		F_test[i][j]= math.exp(-math.pow(dist,2)/math.pow(2*sigma,2))


prediction= np.dot(F_test,W)
prediction= 0.5*(np.sign(prediction-0.5)+1)

score= accuracy_score(prediction,y_test)
print(score.mean())

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


0.896112600536193


Τα συγκεντρωτικά αποτελέσματα είναι:

i. 0.8876675603217158, για k=8

ii. 0.8912198391420911, για k=10

iii. 0.896112600536193, για k=50

Συνεπώς, παρατηρούμε ότι καθώς αυξάνεται το k αυξάνεται και η ακρίβεια (accuracy) των αποτελεσμάτων μας. Η αύξηση αυτή δεν είναι πολύ υψήλη. Ωστόσο, πρέπει να την τονισούμε διότι κάθε ποσοστό αύξησης της ακρίβειας των αποτελεσμάτων μας, είναι πολύ σημαντικό για την εγκυρότητα και την αξιοπιστία των μοντέλων μας.